### Read in .csv files to begin removing/adding any model-specific columns

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [2]:
flights = pd.read_csv('flights_.csv', index_col=0) # the latest flight data, with feature_engineered columns
#passengers = pd.read_csv('passengers_jan-dec_18-19.csv') # the latest passenger data

C:\Users\jsusn\anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,day_of_the_week,origin_wind_speed,origin_visibility,origin_conditions,dest_wind_speed,dest_visibility,dest_conditions,holiday,year,month,scheduled_flight_hour_of_day
60659,2018-01-01,UA,UA,UA,745,UA,N848UA,745,12953,LGA,new york,11292,DEN,denver,1453,1440.0,-13.0,15.0,1455.0,1643.0,7.0,1726,1650.0,-36.0,0.0,NaN,0.0,273.0,250.0,228.0,1620.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,14.8,9.9,Clear,12.3,9.1,Partially cloudy,True,2018,1,14
23125,2018-01-01,UA,UA_CODESHARE,UA,3582,YX,N649RW,3582,11298,DFW,dallas/fort worth,11292,DEN,denver,2016,2005.0,-11.0,13.0,2018.0,2102.0,7.0,2130,2109.0,-21.0,0.0,NaN,0.0,134.0,124.0,104.0,641.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,15.0,9.9,Partially cloudy,12.3,9.1,Partially cloudy,True,2018,1,20
3646,2018-01-01,UA,UA,UA,1591,UA,N87527,1591,11298,DFW,dallas/fort worth,12892,LAX,los angeles,545,539.0,-6.0,17.0,556.0,656.0,24.0,734,720.0,-14.0,0.0,NaN,0.0,229.0,221.0,180.0,1235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,15.0,9.9,Partially cloudy,5.6,4.2,Clear,True,2018,1,5
73558,2018-01-01,WN,WN,WN,5500,WN,N8685B,5500,11292,DEN,denver,10397,ATL,atlanta,1820,1818.0,-2.0,9.0,1827.0,2239.0,8.0,2310,2247.0,-23.0,0.0,NaN,0.0,170.0,149.0,132.0,1199.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,12.3,9.1,Partially cloudy,14.6,9.9,Partially cloudy,True,2018,1,18
77601,2018-01-01,WN,WN,WN,415,WN,N8690A,415,12892,LAX,los angeles,13232,MDW,chicago,1225,1226.0,1.0,7.0,1233.0,1758.0,4.0,1820,1802.0,-18.0,0.0,NaN,0.0,235.0,216.0,205.0,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,5.6,4.2,Clear,12.7,9.9,Clear,True,2018,1,12


### One-Hot encoding for:
- weather
- origin_city_name and dest_city_name
- day_of_week

In [4]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'first_dep_time', 'total_add_gtime', 'longest_add_gtime',
       'day_of_the_week', 'origin_wind_speed', 'origin_visibility',
       'origin_conditions', 'dest_wind_speed', 'dest_visibility',
       'dest_conditions', 'holiday', 'year', 'month',
       'scheduled_flight_hour_of_day'],
      dtype='object')

# After reviewing the 2nd and 3rd models, and how neither improved our r^2 or RMSE, we're going to combine both of them and see what happens

In [5]:
# # We want to change "Rain, Overcast" to Rain = 1, Cloudy = 1
# flights['origin_rain'] = 0
# flights['origin_cloudy'] = 0
# flights['origin_snow'] = 0
# flights['dest_rain'] = 0
# flights['dest_cloudy'] = 0
# flights['dest_snow'] = 0

# # go through each condition, setting all 3 relevant columns
# # there is probably a better way but i know this will work for now...

# flights.loc[flights['origin_conditions'] == 'Partially cloudy', 'origin_rain'] = 0
# flights.loc[flights['origin_conditions'] == 'Partially cloudy', 'origin_cloudy'] = 0.5
# flights.loc[flights['origin_conditions'] == 'Partially cloudy', 'origin_snow'] = 0
# flights.loc[flights['dest_conditions'] == 'Partially cloudy', 'dest_rain'] = 0
# flights.loc[flights['dest_conditions'] == 'Partially cloudy', 'dest_cloudy'] = 0.5
# flights.loc[flights['dest_conditions'] == 'Partially cloudy', 'dest_snow'] = 0

# flights.loc[flights['origin_conditions'] == 'Clear', 'origin_rain'] = 0
# flights.loc[flights['origin_conditions'] == 'Clear', 'origin_cloudy'] = 0
# flights.loc[flights['origin_conditions'] == 'Clear', 'origin_snow'] = 0
# flights.loc[flights['dest_conditions'] == 'Clear', 'dest_rain'] = 0
# flights.loc[flights['dest_conditions'] == 'Clear', 'dest_cloudy'] = 0
# flights.loc[flights['dest_conditions'] == 'Clear', 'dest_snow'] = 0

# flights.loc[flights['origin_conditions'] == 'Rain, Overcast', 'origin_rain'] = 1
# flights.loc[flights['origin_conditions'] == 'Rain, Overcast', 'origin_cloudy'] = 1
# flights.loc[flights['origin_conditions'] == 'Rain, Overcast', 'origin_snow'] = 0
# flights.loc[flights['dest_conditions'] == 'Rain, Overcast', 'dest_rain'] = 1
# flights.loc[flights['dest_conditions'] == 'Rain, Overcast', 'dest_cloudy'] = 1
# flights.loc[flights['dest_conditions'] == 'Rain, Overcast', 'dest_snow'] = 0

# flights.loc[flights['origin_conditions'] == 'Rain, Partially cloudy', 'origin_rain'] = 1
# flights.loc[flights['origin_conditions'] == 'Rain, Partially cloudy', 'origin_cloudy'] = 0.5
# flights.loc[flights['origin_conditions'] == 'Rain, Partially cloudy', 'origin_snow'] = 0
# flights.loc[flights['dest_conditions'] == 'Rain, Partially cloudy', 'dest_rain'] = 1
# flights.loc[flights['dest_conditions'] == 'Rain, Partially cloudy', 'dest_cloudy'] = 0.5
# flights.loc[flights['dest_conditions'] == 'Rain, Partially cloudy', 'dest_snow'] = 0

# flights.loc[flights['origin_conditions'] == 'Overcast', 'origin_rain'] = 0
# flights.loc[flights['origin_conditions'] == 'Overcast', 'origin_cloudy'] = 1
# flights.loc[flights['origin_conditions'] == 'Overcast', 'origin_snow'] = 0
# flights.loc[flights['dest_conditions'] == 'Overcast', 'dest_rain'] = 0
# flights.loc[flights['dest_conditions'] == 'Overcast', 'dest_cloudy'] = 1
# flights.loc[flights['dest_conditions'] == 'Overcast', 'dest_snow'] = 0

# flights.loc[flights['origin_conditions'] == 'Snow, Partially cloudy', 'origin_rain'] = 0
# flights.loc[flights['origin_conditions'] == 'Snow, Partially cloudy', 'origin_cloudy'] = 0.5
# flights.loc[flights['origin_conditions'] == 'Snow, Partially cloudy', 'origin_snow'] = 1
# flights.loc[flights['dest_conditions'] == 'Snow, Partially cloudy', 'dest_rain'] = 0
# flights.loc[flights['dest_conditions'] == 'Snow, Partially cloudy', 'dest_cloudy'] = 0.5
# flights.loc[flights['dest_conditions'] == 'Snow, Partially cloudy', 'dest_snow'] = 1

# flights.loc[flights['origin_conditions'] == 'Rain', 'origin_rain'] = 1
# flights.loc[flights['origin_conditions'] == 'Rain', 'origin_cloudy'] = 0
# flights.loc[flights['origin_conditions'] == 'Rain', 'origin_snow'] = 0
# flights.loc[flights['dest_conditions'] == 'Rain', 'dest_rain'] = 1
# flights.loc[flights['dest_conditions'] == 'Rain', 'dest_cloudy'] = 0
# flights.loc[flights['dest_conditions'] == 'Rain', 'dest_snow'] = 0

# flights.loc[flights['origin_conditions'] == 'Snow, Overcast', 'origin_rain'] = 0
# flights.loc[flights['origin_conditions'] == 'Snow, Overcast', 'origin_cloudy'] = 1
# flights.loc[flights['origin_conditions'] == 'Snow, Overcast', 'origin_snow'] = 1
# flights.loc[flights['dest_conditions'] == 'Snow, Overcast', 'dest_rain'] = 0
# flights.loc[flights['dest_conditions'] == 'Snow, Overcast', 'dest_cloudy'] = 1
# flights.loc[flights['dest_conditions'] == 'Snow, Overcast', 'dest_snow'] = 1

# flights.loc[flights['origin_conditions'] == 'Snow', 'origin_rain'] = 0
# flights.loc[flights['origin_conditions'] == 'Snow', 'origin_cloudy'] = 0
# flights.loc[flights['origin_conditions'] == 'Snow', 'origin_snow'] = 1
# flights.loc[flights['dest_conditions'] == 'Snow', 'dest_rain'] = 0
# flights.loc[flights['dest_conditions'] == 'Snow', 'dest_cloudy'] = 0
# flights.loc[flights['dest_conditions'] == 'Snow', 'dest_snow'] = 1

# flights = flights.drop(columns=['origin_conditions', 'dest_conditions'])

### Key model columns

In [6]:
flights['holiday'] = flights['holiday'].astype(int)

In [18]:
dest_conditions = pd.get_dummies(flights['dest_conditions'], prefix='dest')
origin_conditions = pd.get_dummies(flights['origin_conditions'], prefix='origin')

origin_airport_id = pd.get_dummies(flights['origin_airport_id'], prefix='origin')
dest_airport_id = pd.get_dummies(flights['dest_airport_id'], prefix='dest')

day_of_week = pd.get_dummies(flights['day_of_the_week'])

### We're going to try and use the previous year's route averages for each route, for the month, as a column in the current year
    NOTES: 
    - This means we cannot use 2018 data as we do not have 2017 data to fill in this column.
    - This is a column we will have to add in to the 2020 test set as well.

In [8]:
# NOTE************************
# I comment this out because this step takes a while, 
# I believe there's a better way to implement this but for now this works
#

route_info = []
sep = '_'

# iterate over the entire dataframe, constructing the route_info
for i in range(len(flights[['origin_airport_id', 'dest_airport_id']])):
    route_info.append(str(flights.iloc[i].origin_airport_id) + sep + str(flights.iloc[i].dest_airport_id))

flights['route_info'] = pd.DataFrame(route_info)

In [9]:
flights['route_info']

60659     11057_13930
23125     11278_13930
3646      13930_12953
73558     11278_11298
77601     12264_12892
             ...     
140619    10397_12266
106829    12266_12892
106104    10397_11057
103182    14747_12892
170119    12266_11057
Name: route_info, Length: 207272, dtype: object

In [10]:
# create 4 separate dataframes, one for each year/month

df_filter = (flights['month'] == 12) & (flights['year'] == 2018)
flights_dec_18 = flights[df_filter]

df_filter = (flights['month'] == 1) & (flights['year'] == 2018)
flights_jan_18 = flights[df_filter]

df_filter = (flights['month'] == 12) & (flights['year'] == 2019)
flights_dec_19 = flights[df_filter]

df_filter = (flights['month'] == 1) & (flights['year'] == 2019)
flights_jan_19 = flights[df_filter]

In [11]:
# CREATE LIST OF ALL ROUTES (IN OUR DATASET)
route_info_list = []

for i in flights['route_info'].unique():
    route_info_list.append(i)

#print(route_info_list)
print(f'There are {len(route_info_list)} routes total.')

There are 156 routes total.


In [12]:
# Calculate the historical_arr_delay for each route, for each month, for each year
historical_arr_delay = {}

# GET HISTORICAL_ARR_DELAY for DECEMBER 2018
for index, val in enumerate(route_info_list):
    df_filter = flights_dec_18['route_info'] == str(route_info_list[index])
    historical_arr_delay[val] = flights_dec_18[df_filter]['arr_delay'].mean()

# Set the DECEMBER 2019 VALUES
for k in historical_arr_delay:
    flights_dec_19.loc[flights_dec_19['route_info'] == k, 'historical_arr_delay'] = historical_arr_delay[k]


C:\Users\jsusn\anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jsusn\anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [13]:
historical_arr_delay = {}

# GET HISTORICAL_ARR_DELAY for JANUARY 2018
for index, val in enumerate(route_info_list):
    df_filter = flights_jan_18['route_info'] == str(route_info_list[index])
    historical_arr_delay[val] = flights_jan_18[df_filter]['arr_delay'].mean()

# Set the DECEMBER 2019 VALUES
for k in historical_arr_delay:
    flights_jan_19.loc[flights_jan_19['route_info'] == k, 'historical_arr_delay'] = historical_arr_delay[k]

In [14]:
flights_18 = pd.concat([flights_dec_18, flights_jan_18])

In [15]:
# construct the new dataframe 
flights_19 = pd.concat([flights_dec_19, flights_jan_19])
flights_19

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,day_of_the_week,origin_wind_speed,origin_visibility,origin_conditions,dest_wind_speed,dest_visibility,dest_conditions,holiday,year,month,scheduled_flight_hour_of_day,route_info,historical_arr_delay
167072,2019-12-01,DL,DL,DL,998,DL,N118DU,998,11298,DFW,dallas/fort worth,12953,LGA,new york,705,659.0,-6.0,17.0,716.0,1103.0,5.0,1134,1108.0,-26.0,0.0,NaN,0.0,209.0,189.0,167.0,1389.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,19.7,9.9,Clear,19.7,7.3,"Rain, Partially cloudy",0,2019,12,7,13930_10397,1.472914
114438,2019-12-01,UA,UA,UA,1093,UA,N479UA,1093,11292,DEN,denver,12892,LAX,los angeles,811,809.0,-2.0,12.0,821.0,925.0,6.0,939,931.0,-8.0,0.0,NaN,0.0,148.0,142.0,124.0,862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,10.4,9.9,"Snow, Partially cloudy",3.9,9.9,Rain,0,2019,12,8,14747_13930,0.467890
173908,2019-12-01,AA,AA,AA,2752,AA,N934AN,2752,11298,DFW,dallas/fort worth,11278,DCA,washington,1913,1911.0,-2.0,25.0,1936.0,2251.0,9.0,2300,2300.0,0.0,0.0,NaN,0.0,167.0,169.0,135.0,1192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,19.7,9.9,Clear,11.4,7.4,"Rain, Overcast",0,2019,12,19,10397_11292,2.476404
141851,2019-12-01,UA,UA_CODESHARE,UA,3550,YX,N739YX,3550,12266,IAH,houston,11278,DCA,washington,701,653.0,-8.0,23.0,716.0,1048.0,5.0,1100,1053.0,-7.0,0.0,NaN,0.0,179.0,180.0,152.0,1208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,15.8,9.9,Clear,11.4,7.4,"Rain, Overcast",0,2019,12,7,12892_14747,-0.141509
137110,2019-12-01,AA,AA,AA,1152,AA,N768US,1152,11057,CLT,charlotte,12266,IAH,houston,1745,1903.0,78.0,25.0,1928.0,2051.0,19.0,1935,2110.0,95.0,0.0,NaN,0.0,170.0,187.0,143.0,912.0,78.0,0.0,17.0,0.0,0.0,NaN,NaN,NaN,Sunday,21.5,6.8,"Rain, Partially cloudy",15.8,9.9,Clear,0,2019,12,17,11298_11292,2.751553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153647,2019-01-31,DL,DL,DL,1752,DL,N364NB,1752,10397,ATL,atlanta,12266,IAH,houston,941,937.0,-4.0,44.0,1021.0,1111.0,5.0,1053,1116.0,23.0,0.0,NaN,0.0,132.0,159.0,110.0,689.0,0.0,0.0,23.0,0.0,0.0,NaN,NaN,NaN,Thursday,8.1,9.8,Clear,9.3,6.7,"Rain, Overcast",0,2019,1,9,11057_13930,-1.626632
151184,2019-01-31,B6,B6,B6,23,B6,N946JB,23,12478,JFK,new york,12892,LAX,los angeles,540,534.0,-6.0,24.0,558.0,853.0,11.0,904,904.0,0.0,0.0,NaN,0.0,384.0,390.0,355.0,2475.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,18.3,9.9,Rain,15.0,7.6,"Rain, Partially cloudy",0,2019,1,5,13930_11292,2.485714
137756,2019-01-31,WN,WN,WN,1183,WN,N763SW,1183,13232,MDW,chicago,11278,DCA,washington,1545,1812.0,147.0,24.0,1836.0,2056.0,6.0,1820,2102.0,162.0,0.0,NaN,0.0,95.0,110.0,80.0,601.0,121.0,0.0,15.0,0.0,26.0,NaN,NaN,NaN,Thursday,12.5,7.5,"Rain, Partially cloudy",10.0,9.9,Snow,0,2019,1,15,11292_12892,0.053009
130137,2019-01-31,WN,WN,WN,190,WN,N8607M,190,12191,HOU,houston,11278,DCA,washington,745,738.0,-7.0,10.0,748.0,1108.0,4.0,1140,1112.0,-28.0,0.0,NaN,0.0,175.0,154.0,140.0,1218.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,9.3,6.7,"Rain, Overcast",10.0,9.9,Snow,0,2019,1,7,12478_10397,3.315353


# Build the dataframe to model

In [234]:
# include NEW weather columns & historical_arr_delay
df_flights = pd.concat((
    day_of_week, 
    origin_airport_id, 
    origin_conditions, 
    #flights_19['origin_rain'], 
    #flights_19['origin_cloudy'], 
    #flights_19['origin_snow'], 
    flights_19['origin_visibility'], 
    flights_19['origin_wind_speed'],
    dest_airport_id, 
    dest_conditions, 
    #flights_19['dest_rain'], 
    #flights_19['dest_cloudy'], 
    #flights_19['dest_snow'], 
    flights_19['dest_visibility'], 
    flights_19['dest_wind_speed'], 
    flights_19['holiday'], 
    flights_19['scheduled_flight_hour_of_day'], 
    flights_19['distance'],
    #flights_19['historical_arr_delay'],
    flights_19['arr_delay'] # target variable
), axis=1)

df_flights = df_flights.dropna()

In [235]:
# df_flights = pd.concat((
#     day_of_week, 
#     origin_airport_id, 
#     origin_conditions, 
#     flights_19['origin_visibility'], flights_19['origin_wind_speed'], 
#     dest_airport_id, 
#     dest_conditions, 
#     flights_19['dest_visibility'], flights_19['dest_wind_speed'], 
#     flights_19['holiday'], 
#     flights_19['scheduled_flight_hour_of_day'], 
#     flights_19['distance'],
#     flights_19['historical_arr_delay'], # this notebook's variation
#     flights_19['arr_delay'] # target variable
# ), axis=1)

# df_flights = df_flights.dropna()

In [236]:
# normalize arr_delay, 
# remove some weather columns, 
# change origin_city_name to airport_id, 
# add in hour of day, 
# remove year 
# remove month
# add distance
#
#

In [237]:
flights['arr_delay'].value_counts()[:100]

-14.0     5292
-11.0     5266
-10.0     5263
-12.0     5254
-13.0     5213
          ... 
 99.0       89
 97.0       86
 108.0      86
 104.0      83
 100.0      82
Name: arr_delay, Length: 154, dtype: int64

In [238]:
# the top 100 values in calue_counts sums up to roughly 97% of all data, which is 3 standard deviations worth
# drop anything else

# this change did NOT help the model, it made it significantly worse
#flights = flights.drop(flights[flights.arr_delay < 82].index)

# Linear Regression
### Model-Specific

In [239]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample,shuffle
from sklearn import preprocessing

from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

import xgboost as xgb
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

In [240]:
# Function to calculate VIF
def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

### Dealing with multicollinearity:
https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/

- Dropping one of the one-hot labels to try and avoid "inf." for variance inflation 

In [241]:
calculate_vif(df_flights)

C:\Users\jsusn\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


,Var,Vif
0,Friday,inf
28,"origin_Snow, Overcast",inf
1,Monday,inf
32,dest_10397,inf
33,dest_11057,inf
34,dest_11278,inf
35,dest_11292,inf
36,dest_11298,inf
37,dest_12191,inf
38,dest_12264,inf


In [242]:
columns_to_drop = [
    'Sunday', 
    'origin_10397', 
    'dest_10397', 
    'dest_Clear', 
    'origin_Clear' 
]

In [243]:
df_flights = df_flights.drop(columns=columns_to_drop)

In [244]:
calculate_vif(df_flights)

,Var,Vif
27,origin_visibility,43.96
50,dest_visibility,43.94
28,origin_wind_speed,10.35
51,dest_wind_speed,10.35
53,scheduled_flight_hour_of_day,8.02
54,distance,7.20
52,holiday,1.14
55,arr_delay,1.07
24,origin_Snow,0.11
47,dest_Snow,0.11


In [245]:
X = df_flights.drop(columns=['arr_delay'])
y = df_flights['arr_delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [246]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.3, max_depth = 6, reg_alpha = 0.0001, n_estimators = 100, reg_lambda = 5, random_state = 101)

In [247]:
xg_reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.3, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=101,
             reg_alpha=0.0001, reg_lambda=5, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [248]:
y_preds = xg_reg.predict(X_test)

In [249]:
print(f'R^2: {xg_reg.score(X_test, y_test)}')

R^2: 0.18814347671874876


In [250]:
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE: %f" % (rmse))

RMSE: 43.090315


### Let's try dropping some other columns instead...

In [251]:
# include OLD weather columns & historical_arr_delay
df_flights = pd.concat((
    #day_of_week, 
    #origin_airport_id, 
    #origin_conditions, 
    #flights_19['origin_rain'], 
    #flights_19['origin_cloudy'], 
    #flights_19['origin_snow'], 
    flights_19['origin_visibility'], 
    flights_19['origin_wind_speed'], 
    #dest_airport_id, 
    #dest_conditions, 
    #flights_19['dest_rain'], 
    #flights_19['dest_cloudy'], 
    #flights_19['dest_snow'], 
    flights_19['dest_visibility'], 
    flights_19['dest_wind_speed'], 
    flights_19['holiday'], 
    flights_19['scheduled_flight_hour_of_day'], 
    flights_19['distance'],
    #flights_19['historical_arr_delay'],
    flights_19['arr_delay'] # target variable
), axis=1)

df_flights = df_flights.dropna()

In [252]:
# # include NEW weather columns & historical_arr_delay
# df_flights = pd.concat((
#     #day_of_week, 
#     #origin_airport_id, 
#     origin_conditions, 
#     flights_19['origin_rain'], 
#     flights_19['origin_cloudy'], 
#     flights_19['origin_snow'], 
#     flights_19['origin_visibility'], 
#     flights_19['origin_wind_speed'], 
#     #dest_airport_id, 
#     dest_conditions, 
#     flights_19['dest_rain'], 
#     flights_19['dest_cloudy'], 
#     flights_19['dest_snow'], 
#     flights_19['dest_visibility'], 
#     flights_19['dest_wind_speed'], 
#     flights_19['holiday'], 
#     flights_19['scheduled_flight_hour_of_day'], 
#     flights_19['distance'],
#     #flights_19['historical_arr_delay'],
#     flights_19['arr_delay'] # target variable
# ), axis=1)

# df_flights = df_flights.dropna()

In [253]:
columns_to_drop = [
    #'Tuesday', 
    #'origin_11057', 
    #'dest_11057', 
    #'origin_Partially cloudy', 
    #'dest_Partially cloudy' 
]

In [254]:
df_flights = df_flights.drop(columns=columns_to_drop)

In [255]:
calculate_vif(df_flights)

,Var,Vif
2,dest_visibility,27.72
0,origin_visibility,27.68
5,scheduled_flight_hour_of_day,7.64
1,origin_wind_speed,6.91
3,dest_wind_speed,6.90
6,distance,3.97
4,holiday,1.05
7,arr_delay,1.04


In [256]:
X = df_flights.drop(columns=['arr_delay'])
y = df_flights['arr_delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [257]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.4, max_depth = 6, reg_alpha = 0.0001, n_estimators = 100, reg_lambda = 5, random_state = 101)
xg_reg.fit(X_train, y_train)
y_preds = xg_reg.predict(X_test)
print(f'R^2: {xg_reg.score(X_test, y_test)}')
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE: %f" % (rmse))

R^2: 0.20146723544657963
RMSE: 43.086055
